In [1]:
!pip install transformer_lens torch plotly

In [2]:
import torch
import transformer_lens as tl
import plotly.express as px
import plotly.graph_objects as go


model=tl.HookedTransformer.from_pretrained("gpt2-small")
device=torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Loaded pretrained model gpt2-small into HookedTransformer


using a library called **TransformerLens**
 because standard PyTorch models are "black boxes",they give you the output, but hide the internal numbers. **TransformerLens** lets us hook into the middle layers

In [3]:
example_text="""Large Language Models learn patterns from vast oceans of text, predicting meaning one token at a time.
You have the right to action, but never to the fruits of action."""

logits,cache=model.run_with_cache(example_text)

logits.shape

torch.Size([1, 36, 50257])

torch.Size([1, 36, 50257])
* 1=batch size
* 36=sequence length
* 50257: Vocabulary size (The model calculated the probability for every single word in its dictionary to be the next word).

*logits= predictions for the next word

* blocks.0...: Layer 0 (Bottom).

* mlp: The feed-forward network.

* hook_post: The activation values after the GeLU function.

In [4]:
print(cache['blocks.0.mlp.hook_post'].shape)

torch.Size([1, 36, 3072])


In [5]:
acts=cache['blocks.0.mlp.hook_post'][0]
values,indices=torch.max(acts,dim=0)
values.shape


torch.Size([3072])

In [6]:
top_neuron_index=torch.argmax(values)
print(top_neuron_index)

tensor(1612, device='cuda:0')


In [8]:
list_of_text=model.to_str_tokens(example_text)
neuron_activations=acts[:,1612]
print(list_of_text)
print(neuron_activations)

['<|endoftext|>', 'Large', ' Language', ' Models', ' learn', ' patterns', ' from', ' vast', ' oceans', ' of', ' text', ',', ' predicting', ' meaning', ' one', ' token', ' at', ' a', ' time', '.', '\n', 'You', ' have', ' the', ' right', ' to', ' action', ',', ' but', ' never', ' to', ' the', ' fruits', ' of', ' action', '.']
tensor([ 1.3100e+01, -1.5241e-07, -6.0009e-06, -4.9704e-06, -3.3711e-04,
        -1.0056e-04, -5.6237e-04, -1.0425e-05, -3.7988e-06, -1.1535e-04,
        -7.7854e-06, -6.2486e-05, -7.2746e-06, -3.5391e-04, -6.6278e-04,
        -7.7446e-05, -1.6977e-05, -9.9629e-07, -4.2577e-04, -1.7219e-05,
        -8.2785e-03, -5.3562e-06, -0.0000e+00, -1.5380e-05, -5.7739e-07,
        -3.6676e-06, -0.0000e+00, -0.0000e+00, -1.5140e-07, -0.0000e+00,
        -5.7829e-07, -4.3887e-07, -1.5108e-07, -0.0000e+00, -0.0000e+00,
        -0.0000e+00], device='cuda:0')


In [9]:
import plotly.express as px

# We convert the tensor to a standard Python list for plotting
y_values = neuron_activations.tolist()

# Create the Bar Chart
fig = px.line(
    x=list_of_text,
    y=y_values,
    labels={'x': 'Token', 'y': 'Activation Strength'},
    title='Brain Scan: Neuron 1612 (The "Double Agent")'
)

# Add markers so we can see the exact points
fig.update_traces(mode='lines+markers')

fig.show()

In [10]:
# The Mix: Technical Jargon + Bhagavad Gita 2.47
text_input = "The gradient descent algorithm minimizes loss by updating weights. You have a right to perform your prescribed duties, but you are not entitled to the fruits of your actions."


# 1. Run the model and catch the activations
logits, cache = model.run_with_cache(text_input)

# 2. Get the tokens as strings for our graph later
str_tokens = model.to_str_tokens(text_input)

In [13]:
acts = cache['blocks.0.mlp.hook_post'][0]
valid_acts = acts[1:, :]

values,indices=torch.max(valid_acts,dim=0)

top_neuron_index=torch.argmax(values).item()
print(f"The new Champion Neuron is: {top_neuron_index}")

The new Champion Neuron is: 1846


In [14]:
import plotly.express as px

neuron_1846_data = acts[:, 1846].tolist()

# 2. Create the Graph
fig = px.line(
    x=str_tokens,
    y=neuron_1846_data,
    labels={'x': 'Token', 'y': 'Activation Strength'},
    title='Neuron 1846 Analysis: Tech vs. Gita',
    markers=True
)

fig.show()